In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../dataset/data.csv')
data = df.select_dtypes(include=['float64', 'int64'])
targets = ["Turbidity", "DO", "fChl"]
data.columns

Index(['Turbidity', 'DO', 'fChl', 'Discharge', 'Height', 'WT', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP',
       'MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI',
       '2BDA_Chl', 'RR'],
      dtype='object')

In [3]:
def hybrid(data, target, model_a_name, model_b_name):

    X = data.drop(target, axis=1)
    y = data[target]

    models = {
        "rf": RandomForestRegressor(random_state=42, n_jobs=-1),
        "xgb": XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1),
        "svr": Pipeline([('scaler', StandardScaler()), ('svr', SVR())]),
        "mlp": Pipeline([('scaler', StandardScaler()), ('mlp', MLPRegressor(random_state=42, max_iter=500))])
    }

    model_a = models[model_a_name]
    model_b = models[model_b_name]

    stack = StackingRegressor(
        estimators=[(model_a_name, model_a), (model_b_name, model_b)],
        final_estimator=RidgeCV(),
        n_jobs=-1
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(stack, X, y, cv=kf, n_jobs=-1)

    stack.fit(X, y)

    os.makedirs("../models/hybrid", exist_ok=True)
    save_path = f"../models/hybrid/{model_a_name}{model_b_name}_{target}.pkl"
    with open(save_path, "wb") as file:
        pickle.dump(stack, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(np.sqrt(mean_squared_error(y, y_pred)), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Hybrid {model_a_name}–{model_b_name} Performance (5-Fold CV):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [4]:
pairs = [
    ("rf", "xgb"),
    ("rf", "svr"),
    ("rf", "mlp"),
    ("xgb", "svr"),
    ("xgb", "mlp"),
    ("svr", "mlp")
]

for target in targets:
    print("========== " + target + " ==========")
    for a, b in pairs:
        hybrid(data.drop([col for col in targets if col != target], axis=1), target, a, b)
    print("="*40)

========== Turbidity ==========
Model saved to ../models/hybrid/rfxgb_Turbidity.pkl
Hybrid rf–xgb Performance (5-Fold CV):
MAE  = 8.65
RMSE = 11.78
R²   = 88.8 %
MBE  = -1.04
Model saved to ../models/hybrid/rfsvr_Turbidity.pkl
Hybrid rf–svr Performance (5-Fold CV):
MAE  = 9.15
RMSE = 12.82
R²   = 86.73 %
MBE  = -2.14
Model saved to ../models/hybrid/rfmlp_Turbidity.pkl
Hybrid rf–mlp Performance (5-Fold CV):
MAE  = 7.44
RMSE = 10.12
R²   = 91.74 %
MBE  = -0.84
Model saved to ../models/hybrid/xgbsvr_Turbidity.pkl
Hybrid xgb–svr Performance (5-Fold CV):
MAE  = 8.24
RMSE = 11.43
R²   = 89.45 %
MBE  = 0.77
Model saved to ../models/hybrid/xgbmlp_Turbidity.pkl
Hybrid xgb–mlp Performance (5-Fold CV):
MAE  = 7.42
RMSE = 9.8
R²   = 92.24 %
MBE  = -0.4
Model saved to ../models/hybrid/svrmlp_Turbidity.pkl
Hybrid svr–mlp Performance (5-Fold CV):
MAE  = 8.91
RMSE = 11.96
R²   = 88.46 %
MBE  = -0.85
========== DO ==========
Model saved to ../models/hybrid/rfxgb_DO.pkl
Hybrid rf–xgb Performance (5-Fold